<div style="border-radius: 25px; border: 3px solid #00FFCC; padding: 20px; background: linear-gradient(135deg, #000000, #001f1f); text-align: center; box-shadow: 0 4px 12px rgba(0, 255, 204, 0.4);">
  <h1 style="color: #00FFCC; text-shadow: 2px 2px 6px rgba(0, 255, 204, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 30px;">
    🔊 <br>
    <span style="color: #FF69B4;">📊 Toxic_Chat_Detection</span> Using DistillBERT MODEL <span style="color: #87CEEB;"> CPU Oriented</span>:<br><br>
      <span style="color: #FFD700;">A basic Test Based Chat Classification NLP Model</span> <span style="color: #87CEEB;">Applying The concepts of DistillBert,Tokenizers and DistilBertForSequenceClassification</span>
  </h1>
</div>

In [6]:
!pip install -q transformers==4.39.3 tokenizers==0.15.2 accelerate scikit-learn gradio

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

<div style="border-radius: 25px; border: 3px solid #00FFCC; padding: 20px; background: linear-gradient(135deg, #000000, #001f1f); text-align: center; box-shadow: 0 4px 12px rgba(0, 255, 204, 0.4);">
  <h1 style="color: #00FFCC; text-shadow: 2px 2px 6px rgba(0, 255, 204, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 30px;">
    🔊<br>
    <span style="color: #FF69B4;">📊 Using a Toxic Comment Dataset for types of text - </span> Toxic, Severe Toxic, Obsecene, Identity Hate and Threat<span style="color: #87CEEB;"> </span><br><br>
  </h1>
</div>

<div style="border-radius: 25px; border: 3px solid #00FFCC; padding: 20px; background: linear-gradient(135deg, #000000, #001f1f); text-align: center; box-shadow: 0 4px 12px rgba(0, 255, 204, 0.4);">
  <h1 style="color: #00FFCC; text-shadow: 2px 2px 6px rgba(0, 255, 204, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 30px;">
    <br>
    <span style="color: #FF69B4;">📊 Data Preparation </span><span style="color: #87CEEB;"></span><br><br>
  </h1>
</div>

<div style="border-radius: 25px; border: 3px solid #00FFCC; padding: 20px; background: linear-gradient(135deg, #000000, #001f1f); text-align: center; box-shadow: 0 4px 12px rgba(0, 255, 204, 0.4);">
  <h1 style="color: #00FFCC; text-shadow: 2px 2px 6px rgba(0, 255, 204, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 30px;">
    <br>
    <span style="color: #FF69B4;">📊 Taking Data Input With Priortising The types</span> Threat, Toxic, Obscene <span style="color: #87CEEB;">Identity_Hate, Severe_Toxic</span><br><br>
  </h1>
</div>

In [7]:
df = pd.read_csv("/kaggle/input/toxic-chats/train.csv", engine="python", on_bad_lines="skip")
df = df[['comment_text','toxic','severe_toxic','obscene','threat','insult','identity_hate']]
df.shape


(159571, 7)

<div style="border-radius: 25px; border: 3px solid #00FFCC; padding: 20px; background: linear-gradient(135deg, #000000, #001f1f); text-align: center; box-shadow: 0 4px 12px rgba(0, 255, 204, 0.4);">
  <h1 style="color: #00FFCC; text-shadow: 2px 2px 6px rgba(0, 255, 204, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 30px;">
    <br>
    <span style="color: #FF69B4;">📊 Sample & Split </span><span style="color: #87CEEB;">The original dataset is huge. We take a smaller chunk to make it run faster on your laptop, then split it into "Study Material" (Train) and "Exam Material" (Validation)</span><br><br>
  </h1>
</div>

In [8]:
df_small = df.sample(74980, random_state=42).reset_index(drop=True)

label_cols = df_small.columns[1:].tolist()
train_df, val_df = train_test_split(df_small, test_size=0.1, random_state=42)

<div style="border-radius: 25px; border: 3px solid #00FFCC; padding: 20px; background: linear-gradient(135deg, #000000, #001f1f); text-align: center; box-shadow: 0 4px 12px rgba(0, 255, 204, 0.4);">
  <h1 style="color: #00FFCC; text-shadow: 2px 2px 6px rgba(0, 255, 204, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 30px;">
    <br>
    <span style="color: #FF69B4;">📊  The Custom Dataset (Crucial OOP Part) </span><span style="color: #87CEEB;"> This is the bridge between raw text and the AI. It converts sentences into numbers (Tokens)</span><br><br>
  </h1>
</div>

In [9]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

class ToxicDataset(Dataset):
    def __init__(self, df, max_len=96):
        self.texts = df['comment_text'].tolist()
        self.labels = df[label_cols].values.astype('float32')
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = tokenizer(
            str(self.texts[idx]),
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx])
        }

train_ds = ToxicDataset(train_df)
val_ds   = ToxicDataset(val_df)

train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=32, shuffle=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

<div style="border-radius: 25px; border: 3px solid #00FFCC; padding: 20px; background: linear-gradient(135deg, #000000, #001f1f); text-align: center; box-shadow: 0 4px 12px rgba(0, 255, 204, 0.4);">
  <h1 style="color: #00FFCC; text-shadow: 2px 2px 6px rgba(0, 255, 204, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 30px;">
    <br>
    <span style="color: #FF69B4;">📊 Handling Imbalance (The "Pos Weight")</span><span style="color: #87CEEB;"> Real life has few toxic comments. This calculates a "Fairness Score" to punish the model more if it misses a toxic comment.</span><br><br>
  </h1>
</div>

In [10]:
pos_counts = train_df[label_cols].sum().values
neg_counts = len(train_df) - pos_counts

ratio = neg_counts / pos_counts
ratio_clipped = np.clip(ratio, 1.0, 5.0)

pos_weight = torch.tensor(ratio_clipped, dtype=torch.float32).to(device)
print("pos_weight:", dict(zip(label_cols, ratio_clipped)))


pos_weight: {'toxic': 5.0, 'severe_toxic': 5.0, 'obscene': 5.0, 'threat': 5.0, 'insult': 5.0, 'identity_hate': 5.0}


<div style="border-radius: 25px; border: 3px solid #00FFCC; padding: 20px; background: linear-gradient(135deg, #000000, #001f1f); text-align: center; box-shadow: 0 4px 12px rgba(0, 255, 204, 0.4);">
  <h1 style="color: #00FFCC; text-shadow: 2px 2px 6px rgba(0, 255, 204, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 30px;">
    <br>
    <span style="color: #FF69B4;">📊 The Model</span><span style="color: #87CEEB;"> Downloads a pre-trained "DistilBERT" brain and sets it up for our specific task (6-class classification)</span><br><br>
  </h1>
</div>

In [11]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label_cols)
).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from tqdm.auto import tqdm

def train_epoch():
    model.train()
    total = 0
    for batch in train_dl:
        optimizer.zero_grad()
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        out = model(ids, attention_mask=mask).logits
        loss = criterion(out, labels)

        loss.backward()
        optimizer.step()
        total += loss.item()
    return total/len(train_dl)

@torch.no_grad()
def val_epoch():
    model.eval()
    total = 0
    probs=[]; true=[]
    for batch in val_dl:
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        out = model(ids, attention_mask=mask).logits
        total += criterion(out, labels).item()
        probs.append(torch.sigmoid(out).cpu())
        true.append(labels.cpu())

    return total/len(val_dl), torch.cat(true), torch.cat(probs)


<div style="border-radius: 25px; border: 3px solid #00FFCC; padding: 20px; background: linear-gradient(135deg, #000000, #001f1f); text-align: center; box-shadow: 0 4px 12px rgba(0, 255, 204, 0.4);">
  <h1 style="color: #00FFCC; text-shadow: 2px 2px 6px rgba(0, 255, 204, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 30px;">
    <br>
    <span style="color: #FF69B4;">📊 The Loop (Training)</span><span style="color: #87CEEB;"> The actual learning process.</span><br><br>
  </h1>
</div>

In [13]:
for e in range(3):
    print(f"\n🔥 Epoch {e+1}/3")
    tr_loss = train_epoch()
    val_loss, val_labels, val_probs = val_epoch()
    print(f"Train Loss: {tr_loss:.4f} | Val Loss: {val_loss:.4f}")



🔥 Epoch 1/3
Train Loss: 0.1322 | Val Loss: 0.0996

🔥 Epoch 2/3
Train Loss: 0.0869 | Val Loss: 0.0986

🔥 Epoch 3/3
Train Loss: 0.0647 | Val Loss: 0.1201


<div style="border-radius: 25px; border: 3px solid #00FFCC; padding: 20px; background: linear-gradient(135deg, #000000, #001f1f); text-align: center; box-shadow: 0 4px 12px rgba(0, 255, 204, 0.4);">
  <h1 style="color: #00FFCC; text-shadow: 2px 2px 6px rgba(0, 255, 204, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 30px;">
    <br>
    <span style="color: #FF69B4;">📊 Threshold Optimization (Advanced)</span><span style="color: #87CEEB;">  Most models assume anything > 50% is "True". But maybe for "Threats", we want to be safer and flag anything > 20%. This block finds the perfect "Cut-off Score" for each category</span><br><br>
  </h1>
</div>

In [14]:
best_th = {}
for i, col in enumerate(label_cols):
    y = val_labels[:, i].numpy()
    p = val_probs[:, i].numpy()
    best_f1 = 0
    best_t = 0.5
    for t in np.linspace(0.1,0.9,17):
        f = f1_score(y,(p>=t).astype(int),zero_division=0)
        if f > best_f1:
            best_f1 = f
            best_t = t
    best_th[col] = best_t
best_th


{'toxic': 0.9,
 'severe_toxic': 0.65,
 'obscene': 0.75,
 'threat': 0.8,
 'insult': 0.55,
 'identity_hate': 0.85}

<div style="border-radius: 25px; border: 3px solid #00FFCC; padding: 20px; background: linear-gradient(135deg, #000000, #001f1f); text-align: center; box-shadow: 0 4px 12px rgba(0, 255, 204, 0.4);">
  <h1 style="color: #00FFCC; text-shadow: 2px 2px 6px rgba(0, 255, 204, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 30px;">
    <br>
    <span style="color: #FF69B4;">📊 The Prediction Logic</span><span style="color: #87CEEB;">  This is the function you use in the real world. It takes raw text and gives a human-readable answer.</span><br><br>
  </h1>
</div>

In [15]:
PRIORITY = ["threat", "identity_hate", "obscene", "insult", "toxic","severe_toxic"]

def predict_raw(text):
    model.eval()
    with torch.no_grad():
        enc = tokenizer(text, return_tensors="pt",
                        truncation=True, padding="max_length", max_length=96)
        logits = model(**{k:v.to(device) for k,v in enc.items()}).logits
        probs = torch.sigmoid(logits).cpu().numpy()[0]

    out = {}
    for i, col in enumerate(label_cols):
        p = float(probs[i])
        th = float(best_th[col])
        out[col] = {
            "prob": p,
            "threshold": th,
            "over": p - th,
            "active": p >= th
        }
    return out

def choose_main_label(raw):
    active = [lbl for lbl, info in raw.items() if info["active"]]
    if not active:
        return "Non-Toxic"

    def score(lbl):
        info = raw[lbl]
        return (info["over"], -PRIORITY.index(lbl))

    return max(active, key=score)

def predict_final(text):
    raw = predict_raw(text)
    main = choose_main_label(raw)
    details = {lbl: int(info["active"]) for lbl, info in raw.items()}
    return {"classification": main, "details": details, "raw": raw}


In [16]:
print("T1:", predict_final("You are an amazing person, stay blessed!"))
print("T2:", predict_final("You are stupid and useless"))
print("T3:", predict_final("Fuck off idiot"))
print("T4:", predict_final("I will kill you tomorrow"))
print("T5:", predict_final("you are a black idiot"))


T1: {'classification': 'Non-Toxic', 'details': {'toxic': 0, 'severe_toxic': 0, 'obscene': 0, 'threat': 0, 'insult': 0, 'identity_hate': 0}, 'raw': {'toxic': {'prob': 0.0017085822764784098, 'threshold': 0.9, 'over': -0.8982914177235216, 'active': False}, 'severe_toxic': {'prob': 1.2471877198549919e-05, 'threshold': 0.65, 'over': -0.6499875281228015, 'active': False}, 'obscene': {'prob': 0.000303219334455207, 'threshold': 0.75, 'over': -0.7496967806655448, 'active': False}, 'threat': {'prob': 3.197705154889263e-05, 'threshold': 0.8, 'over': -0.7999680229484512, 'active': False}, 'insult': {'prob': 0.00028353172820061445, 'threshold': 0.55, 'over': -0.5497164682717994, 'active': False}, 'identity_hate': {'prob': 0.00011216305574635044, 'threshold': 0.85, 'over': -0.8498878369442536, 'active': False}}}
T2: {'classification': 'insult', 'details': {'toxic': 1, 'severe_toxic': 0, 'obscene': 1, 'threat': 0, 'insult': 1, 'identity_hate': 0}, 'raw': {'toxic': {'prob': 0.9968859553337097, 'thresh

In [17]:
# import gradio as gr

# def classify_ui(text):
#     out = predict_final(text)
#     return out["classification"]

# demo = gr.Interface(
#     fn=classify_ui,
#     inputs=gr.Textbox(lines=3, placeholder="Enter a comment..."),
#     outputs="text",
#     title="ToxiGuard v2 – Toxic Comment Detector",
#     description="Detects the primary harmful category in user comments using BERT",
#     allow_flagging="never"
# )

# demo.launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://2d322c51263d0b0fd1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
